In [1]:
import mysql.connector
import pandas as pd

In [2]:
config = {
  'user': "***********",
  'password': "***********",
  'host': '***********',
  'database': "yelp_mexican_food",
  'raise_on_warnings': True
}

In [3]:
# connect to the MySQL database
connection = mysql.connector.connect(**config)

# create a cursor object to execute SQL queries
cursor = connection.cursor()

In [4]:
# view
cursor.execute('SELECT * FROM final_df')
cursor.fetchall()

[(0,
  'taqueria-don-pepe-fresno-2',
  'Taqueria Don Pepe',
  0,
  655,
  'alias: mexican, title: Mexican',
  4.5,
  "['delivery']",
  '$',
  'address1: 4582 N Blackstone Ave, address2: , address3: , city: Fresno, zip_code: 93726, country: US, state: CA, display_address: 4582 N Blackstone Ave, Fresno, CA 93726',
  15592241431.0,
  '(559) 224-1431'),
 (1,
  'cocina-village-fresno',
  'Cocina Village',
  0,
  308,
  'alias: mexican, title: Mexican',
  4.5,
  "['pickup', 'delivery']",
  '$$',
  'address1: 3050 W Shaw Ave, address2: Ste 111, address3: , city: Fresno, zip_code: 93711, country: US, state: CA, display_address: 3050 W Shaw Ave, Ste 111, Fresno, CA 93711',
  15598401663.0,
  '(559) 840-1663'),
 (2,
  'la-enchilada-fresno',
  'La Enchilada',
  0,
  103,
  'alias: mexican, title: Mexican',
  4.5,
  "['pickup', 'delivery']",
  '$',
  'address1: 3848 N Blackstone Ave, address2: , address3: , city: Fresno, zip_code: 93726, country: US, state: CA, display_address: 3848 N Blackstone A

# Cleaning

In [5]:
# update safe settings
cursor.execute("SET SQL_SAFE_UPDATES = 0;")
cursor.fetchall()

[]

In [6]:
# remove duplicates
clean2 = cursor.execute('SELECT DISTINCT * FROM final_df;')
cursor.fetchall()

[(0,
  'taqueria-don-pepe-fresno-2',
  'Taqueria Don Pepe',
  0,
  655,
  'alias: mexican, title: Mexican',
  4.5,
  "['delivery']",
  '$',
  'address1: 4582 N Blackstone Ave, address2: , address3: , city: Fresno, zip_code: 93726, country: US, state: CA, display_address: 4582 N Blackstone Ave, Fresno, CA 93726',
  15592241431.0,
  '(559) 224-1431'),
 (1,
  'cocina-village-fresno',
  'Cocina Village',
  0,
  308,
  'alias: mexican, title: Mexican',
  4.5,
  "['pickup', 'delivery']",
  '$$',
  'address1: 3050 W Shaw Ave, address2: Ste 111, address3: , city: Fresno, zip_code: 93711, country: US, state: CA, display_address: 3050 W Shaw Ave, Ste 111, Fresno, CA 93711',
  15598401663.0,
  '(559) 840-1663'),
 (2,
  'la-enchilada-fresno',
  'La Enchilada',
  0,
  103,
  'alias: mexican, title: Mexican',
  4.5,
  "['pickup', 'delivery']",
  '$',
  'address1: 3848 N Blackstone Ave, address2: , address3: , city: Fresno, zip_code: 93726, country: US, state: CA, display_address: 3848 N Blackstone A

In [7]:
# drop columns
cursor.execute('ALTER TABLE final_df DROP COLUMN phone, DROP COLUMN display_phone;')
cursor.fetchall()

[]

In [8]:
# drop null columns 
cursor.execute('DELETE FROM final_df WHERE name IS NULL OR is_closed IS NULL OR review_count IS NULL OR categories IS NULL OR rating IS NULL OR transactions IS NULL OR price IS NULL;')
cursor.fetchall()

[]

In [9]:
# delete closed locations
cursor.execute('DELETE FROM final_df WHERE is_closed = 1;')
cursor.fetchall()

[]

In [15]:
# view final results
cursor.execute('SELECT * FROM final_df;')
results = cursor.fetchall()

In [16]:
# Convert the results to a DataFrame
df = pd.DataFrame(results)

df.to_csv('clean_yelp_food.csv', index=False)

# EDA

Questions
1. What restaurant has the best ratings
2. What restaurant has the worst ratings
3. Which restaurants offer delivery with a high rating


In [17]:
cursor.execute('SELECT alias, rating FROM final_df ORDER BY rating DESC LIMIT 5;')
sql2 = cursor.fetchall()

In [18]:
# top 5 highest rated restaurants
sql2

[('garrafa-fresno', 5.0),
 ('la-mexicana-bakery-and-taqueria-fresno', 5.0),
 ('julis-taqueria-fresno', 5.0),
 ('taconazo-fresno', 5.0),
 ('tacos-el-pelon-ricardo-torres-fresno', 5.0)]

In [19]:
cursor.execute('SELECT alias, rating FROM final_df ORDER BY rating LIMIT 5;')
sql3 = cursor.fetchall()

In [20]:
# top 5 lowest rated restaurants
sql3

[('jack-in-the-box-fresno-13', 1.0),
 ('salsa-tequila-and-grill-fresno', 1.5),
 ('culichi-town-fresno', 2.0),
 ('arsenios-fresno-9', 2.0),
 ('el-pollo-loco-fresno-7', 2.0)]

In [21]:
cursor.execute("SELECT alias,rating, transactions FROM final_df WHERE transactions LIKE '%delivery%' LIMIT 5;")
sql4 = cursor.fetchall()

In [22]:
# top 5 highest rated restaurants that offer delivery
sql4

[('taqueria-don-pepe-fresno-2', 4.5, "['delivery']"),
 ('cocina-village-fresno', 4.5, "['pickup', 'delivery']"),
 ('la-enchilada-fresno', 4.5, "['pickup', 'delivery']"),
 ('sabor-cocina-latina-and-bar-fresno', 4.5, "['delivery']"),
 ('pico-de-gallo-mexican-food-fresno-2', 4.0, "['pickup', 'delivery']")]

In [24]:
cursor.close()

True